In [69]:
import pandas as pd
import numpy as np
import pickle as p

In [70]:
data_all = pd.read_csv('data_all.csv')

In [71]:
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

In [72]:
item_subset

[202, 16019, 3732, 1230, 3766, 2589, 2531, 1195, 3726, 3743]

### `/train`

In [73]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [74]:
# Model Training Result
r.json()['result']

'Success'

In [75]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,202.0,2110.440190,0.723515,355.472149,0.168435,486.806146
1,1195.0,129.492813,0.638400,27.630060,0.213371,34.985347
2,1230.0,8.996674,0.402763,3.116891,0.346449,4.535508
3,2531.0,318.983568,0.483374,91.194603,0.285891,119.147616
4,2589.0,161.616036,0.695982,34.209791,0.211673,45.069744
5,3726.0,74.383607,0.638035,15.701809,0.211092,20.547442
6,3732.0,126.142506,0.665788,25.081985,0.198838,33.737788
7,3743.0,132.480974,0.651685,33.399662,0.252109,44.757517
8,3766.0,91.448228,0.759059,15.831570,0.173121,21.308304
9,16019.0,109.613186,0.166359,33.442154,0.305092,41.383536


### `/optimize`

In [93]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [94]:
best_prices = r.json()['result']
best_prices

{'2256': 8.81,
 '2257': 11.99,
 '2721': 12.56,
 '3718': 7.42,
 '3722': 10.19,
 '3727': 10.47,
 '3732': 10.56,
 '7009': 6.09,
 '7115': 4.92,
 '7157': 7.12}

In [95]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['pred_q']

In [96]:
qty

{'2256': 79,
 '2257': 102,
 '2721': 140,
 '3718': 486,
 '3722': 1468,
 '3727': 1500,
 '3732': 424,
 '7009': 14902,
 '7115': 100,
 '7157': 27}

In [97]:
# Calculate Revenue
revenue = 0
for item in qty.keys():
    revenue += (best_prices_input[item]*qty[item])
    
# Number is obscenely high due to some outliers,
# Notably 7009, likely due to lack of constraints etc.
print(revenue)

188301.96


### `/predict/`

In [69]:
# Craft mock data input, average historical prices of items
prices_input = {}
for item in item_subset:
    avg_item_price = data_subset.query('Item_ID=={}'.format(item))['Price_'].mean()
    prices_input[str(item)] = round(avg_item_price,2)
    
prices_input

{'3722': 10.72,
 '3718': 7.6,
 '2721': 13.07,
 '2256': 8.27,
 '7009': 4.62,
 '2257': 11.54,
 '3732': 11.79,
 '3727': 10.81,
 '7115': 4.81,
 '7157': 6.66}

In [70]:
import requests
import json

url = 'http://localhost:5000/predict/'



payload = {'prices': prices_input,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [71]:
# What If Prediction
qty = r.json()['pred_q']
qty

{'2256': 57,
 '2257': 18,
 '2721': 26,
 '3718': 291,
 '3722': 361,
 '3727': 288,
 '3732': 150,
 '7009': 1922,
 '7115': 34,
 '7157': 12}

In [72]:
# Calculate Revenue
revenue = 0
for item in qty.keys():
    revenue += (prices_input[item]*qty[item])
    
print(revenue)

21105.329999999998
